# Generating Vegan Recipes

!!! note
    To download this example as a Jupyter notebook, click [here](https://github.com/ShreyaR/guardrails/blob/main/docs/examples/recipe_generation.ipynb).

In this example, we will use Guardrails to generate vegan mac and cheese recipe.

## Objective

We want to generate a vegan Mac-n-Cheese recipe as a list of ingredients and instructions. We will use Guardrails to make sure the recipe is vegan.

In [1]:
import guardrails as gd
from rich import print

## Step 1: Create the RAIL Spec

Ordinarily, we would create an RAIL spec in a separate file. For the purposes of this example, we will create the spec in this notebook as a string following the RAIL syntax. For more information on RAIL, see the [RAIL documentation](rail/output.md).

Here, we request:



In [8]:
rail_str = """
<rail version="0.1">

<script language='python'>
from dataclasses import dataclass
from guardrails.validators import Validator, EventDetail, register_validator

import re
from typing import Dict, List

NON_VEGAN_INGREDIENTS = ["butter", "milk", "eggs", "cheese", "cream", "yogurt"]
SUBSTITUTIONS = {
    "butter": "margarine",
    "milk": "soy milk",
    "eggs": "flax eggs",
    "cheese": "vegan cheese",
    "cream": "soy cream",
    "yogurt": "soy yogurt",
}

@register_validator(name="is-vegan", data_type="string")
class IsVegan(Validator):

    def validate(self, key, value, schema) -> Dict:
        global NON_VEGAN_INGREDIENTS, SUBSTITUTIONS

        # Make sure the ingredient is not in the list of non-vegan ingredients.
        if value.lower() in NON_VEGAN_INGREDIENTS:
            raise EventDetail(
                key,
                value,
                schema,
                f"Value {value} is not vegan.",
                # Programmatically fix the value by replacing it with a vegan
                # substitute.
                SUBSTITUTIONS[value.lower()],
            )

        return schema
</script>


<output>
    <list name="ingredients" description="What are the ingredients for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="name" format="is-vegan" on-fail-is-vegan="fix" />
            <string name="brand" description="Suggested brand for the ingredient (if any)" />
            <bool name="optional" description="Is the ingredient necessary?" />
            <float name="quantity" format="units-imperial" />
            <string name="units" format="units-imperial" />
        </object>
    </list>
    <list name="instructions" description="What are the instructions for the recipe?">
        <object>
            <integer name="index" format="1-indexed" />
            <string name="step" />
        </object>
    </list>
</output>


<prompt>
Generate a recipe for vegan mac and cheese.
@complete_json_suffix
</prompt>

</rail>
"""

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the output of the LLM. This object:

1. Enforces the quality criteria specified in the RAIL spec.
2. Takes corrective action when the quality criteria are not met.
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [9]:
guard = gd.Guard.from_rail_string(rail_str)

/Users/krandiash/Desktop/workspace/projects/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter 1-indexed is not valid for element integer.
  warnings.warn(
/Users/krandiash/Desktop/workspace/projects/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter units-imperial is not valid for element float.
  warnings.warn(
/Users/krandiash/Desktop/workspace/projects/guardrails/guardrails/datatypes.py:53: UserWarning: Formatter units-imperial is not valid for element string.
  warnings.warn(


As we can see, a few formatters weren't supported. These formatters won't be enforced in the output, but this information can still be used to generate a prompt.

We see the prompt that will be sent to the LLM. The `{document}` is substituted with the user provided value at runtime.

In [10]:
print(guard.base_prompt)

Generate a recipe for vegan mac and cheese.

Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <list name="ingredients" description="What are the ingredients for the recipe?">
        <object>
            <integer name="index" format="1-indexed"/>
            <string name="name" format="is-vegan"/>
            <string name="brand" description="Suggested brand for the ingredient (if any)"/>
            <bool name="optional" description="Is the ingredient necessary?"/>
            <float name="quantity" format="units-imperial"/>
            <string name="units" format="units-imperial"/>
        </object>
    </list>
    <list name="instructions" description="What are the instructions for the recipe?">
        <object>
            <integer name="index" format="1-indexed"/>
            <string name="step"/>
        </object>
    </list>
</output>




ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `None`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{{'foo': 'example one'}}`
- `<list name='bar'><string format='upper-case' /></list>` => `{{"bar": ['STRING ONE', 'STRING TWO', etc.]}}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{{'baz': {{'foo': 'Some String', 'index': 1}}}}`

JSON Object:

## Step 3: Wrap the LLM API call with `Guard`

In [11]:
import openai

raw_llm_response, validated_response = guard(
    openai.Completion.create,
    engine='text-davinci-003',
    max_tokens=2048,
    temperature=0
)

The `guard` wrapper returns the raw_llm_respose (which is a simple string), and the validated and corrected output (which is a dictionary).

We can see that the output is a dictionary with the correct schema and types.

In [12]:
print(validated_response)

{
    'ingredients': [
        {'index': 1, 'name': 'macaroni', 'brand': 'Barilla', 'optional': False, 'quantity': 8.0, 'units': 'oz'},
        {
            'index': 2,
            'name': 'vegan butter',
            'brand': 'Earth Balance',
            'optional': False,
            'quantity': 2.0,
            'units': 'tbsp'
        },
        {
            'index': 3,
            'name': 'all-purpose flour',
            'brand': 'Gold Medal',
            'optional': False,
            'quantity': 2.0,
            'units': 'tbsp'
        },
        {'index': 4, 'name': 'vegan milk', 'brand': 'Oatly', 'optional': False, 'quantity': 2.0, 'units': 'cups'},
        {'index': 5, 'name': 'vegan cheese', 'brand': 'Daiya', 'optional': False, 'quantity': 8.0, 'units': 'oz'},
        {
            'index': 6,
            'name': 'nutritional yeast',
            'brand': "Bob's Red Mill",
            'optional': False,
            'quantity': 2.0,
            'units': 'tbsp'
        },
        {
            'index': 7,
            'name': 'garlic powder',
            'brand': 'McCormick',
            'optional': False,
            'quantity': 1.0,
            'units': 'tsp'
        },
        {
            'index': 8,
            'name': 'onion powder',
            'brand': 'McCormick',
            'optional': False,
            'quantity': 1.0,
            'units': 'tsp'
        },
        {'index': 9, 'name': 'salt', 'brand': 'Morton', 'optional': False, 'quantity': 1.0, 'units': 'tsp'},
        {
            'index': 10,
            'name': 'black pepper',
            'brand': 'McCormick',
            'optional': False,
            'quantity': 1.0,
            'units': 'tsp'
        }
    ],
    'instructions': [
        {'index': 1, 'step': 'Preheat oven to 350°F.'},
        {
            'index': 2,
            'step': 'Bring a large pot of salted water to a boil. Add macaroni and cook according to package 
instructions.'
        },
        {'index': 3, 'step': 'Drain macaroni and set aside.'},
        {
            'index': 4,
            'step': 'In a medium saucepan, melt vegan butter over medium heat. Add flour and whisk until combined. 
Cook for 1 minute, stirring constantly.'
        },
        {
            'index': 5,
            'step': 'Slowly add vegan milk, whisking constantly. Cook for 3-4 minutes, stirring constantly, until 
sauce thickens.'
        },
        {
            'index': 6,
            'step': 'Remove from heat and stir in vegan cheese, nutritional yeast, garlic powder, onion powder, 
salt, and pepper. Stir until cheese is melted and sauce is smooth.'
        },
        {'index': 7, 'step': 'Add macaroni to the sauce and stir until combined.'},
        {
            'index': 8,
            'step': 'Transfer macaroni and cheese to a 9x13 inch baking dish. Bake for 20 minutes, or until cheese 
is bubbly and golden brown.'
        },
        {'index': 9, 'step': 'Serve warm and enjoy!'}
    ]
}